In [ ]:
from __future__ import print_function
import os
import subprocess
import time

from astropy.io import ascii
from astropy.io import fits
from astropy.nddata import Cutout2D
from astropy.stats import sigma_clipped_stats
from astropy.table import Table
from astropy import units as u
from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize
from astropy.visualization import astropy_mpl_style

import image_registration

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import

import numpy as np

import pandas as pd
import pandasql as ps

from PIL import Image
import PIL.ImageOps    

import scipy.misc

plt.style.use(astropy_mpl_style)

In [ ]:
fitsImageDir = '/home/mj1e16/moleGazer/Photos/F/fitsImages/16bit/' # directory of reduced fits images
regionDirectory = '/home/mj1e16/moleGazer/Photos/F/fitsImages/16bit/' # location of ds9 region files containing mole regions
outputFileDir = '/home/mj1e16/moleGazer/Photos/F/fitsImages/simMolePos/' # directory to dump output files

In [ ]:
def getImageData(imageName,extension):
    '''
    Extract image data from file name.
    
    Parameters:
        imageName (str): Name of image.
        extension (int): Extension to be extracted.
        
    Returns:
        imageData (array): 2-D numpy array of image data.
    '''
    hdu = fits.open(imageName)
    imageData = hdu[extension].data
    return imageData

In [ ]:
def assessQuality(output,sqlQuery,refPandasNew):
    '''
    Match tables of objects found between two images to determine if the matching was good.
    
    Parameters:
        output (astropy table): Table of objects found in original image.
        sqlQuery (str): SQL command join the two tables.
        refPandasNew (pandas dataframe): Dataframe of objects found via the matching algorithm.
    
    Returns:
        newdf (pandas dataframe): Matched objects the two input sources.
    '''
    pandasTable = Table.to_pandas(output)
    newdf = ps.sqldf(sqlQuery,locals())
    Nrecovered = len(newdf)
    Ntarget = len(refPandasNew)
    Ntot = len(pandasTable)
    return [Nrecovered,Ntarget,Ntot]

In [ ]:
def sextractor(imagename,valList,sextractoryDir='/home/mj1e16/sextractor/sextractor-master/config/',cataloguename='/home/mj1e16/sextractor/sextractor-master/config/moleTable.cat',tableDir='/home/mj1e16/moleGazer/Photos/F/fitsImages/simMolePos/'):
    '''
    Run SExtractor on chosen image.
    
    Parameters:
        imagename (str): Name of target image.
        valList (lst): Chosen input parameters for SExtractor (detection threshold, detection area, filter name).
        sextractoryDir (str): Directory of SExtractor installation.
        cataloguename (str): Name of catalogue produced by SExtractor.
        tableDir (str): Directory to store output tables.
    '''
    
    imTabName = imagename.replace('.fits','')
    loc1 = imTabName.rfind('/') + 1
    imTabName = imTabName[loc1:]+'{}_{}_{}'.format(valList[0],valList[1],valList[2])+'.cat'
    os.chdir(sextractoryDir)
    subprocess.call(['sex',imagename,'-c','default_1.sex'])
    assoc = Table.read(cataloguename,format='ascii.sextractor')
    subprocess.call(['cp',cataloguename,tableDir+imTabName])
    return assoc

In [ ]:
def makeDS9RegFile(sexTabList,fileNameBase,tabType,colour='red',radius='10'):
    '''
    Transforms a list of SExtractor, DAOphot, or IRAF output tables to a ds9 region file.
    
    Parameters:
        sexTabList (list): List of SExtractor output tables.
        fileNameBase (str): Naming convention for ds9 region file.
        tabType (str): Type of output file.
        colour (str): Colour of apertures created in ds9 region file.
        radius (str): Size of apertures created in ds9 region file.
        
    Returns:
        fileName (str): Name of created ds9 region file.
    
    '''
    for tables in range(len(sexTabList)):
        if tabType == 'dao':
            xcoords = sexTabList[tables]['xcentroid']
            ycoords = sexTabList[tables]['ycentroid']
        elif tabType == 'sexMedian':
            ra = sexTabList[tables]['RA']
            dec = sexTabList[tables]['DEC']
        elif tabType == 'sex':
            xcoords = sexTabList[tables]['X_IMAGE']
            ycoords = sexTabList[tables]['Y_IMAGE']
        elif tabType == 'iraf':
            xcoords = sexTabList[tables]['X_POS']
            ycoords = sexTabList[tables]['Y_POS']        
        else:
            print('tab type error')
            break
        bigString = 'image\n'
        for x in range(len(xcoords)):
            bigString += 'circle({},{},'.format(xcoords[x],ycoords[x])+radius+') # color={}\n'.format(colour)
        fileName = fileNameBase + '.reg'
        with open(fileName,'w') as f:
            f.write(bigString)
            
    return fileName

In [ ]:
def makeConfig(valList,tableName='moleTable.cat',defaultDir='/home/mj1e16/sextractor/sextractor-master/config/',attributeList=['DETECT_THRESH','DETECT_MINAREA','FILTER_NAME']):
    '''
    Make SExtractor configuration file with chosen input parameters.
    
    Parameters:
        valList (lst): Values for chosen SExtractor settings.
        tableName (str): Name of table produced by SExtractor.
        defaultDir (str): Directory of SExtractor installation.
        attributeList (list): Names of SExtractor settings to be replace (must be strings and as written in config file).
        
    Returns:
        confName (str): Name of configuration file.
        tableName (str): Name of table produced by SExtractor.
        catName (str): Name of table produced by SExtractor.
    '''
    # catName and tableName are the same in this itteration of the code so catName can be removed
    with open(defaultDir+'/default.sex','r') as f:
        data  = f.read()
    for x in range(len(valList)):
        nameLoc = data.find(attributeList[x]) + len(attributeList[x])
        endLoc = data[nameLoc:].find('#') + nameLoc
        newData = data[:nameLoc] + ' '+str(valList[x])+' ' + data[endLoc:]
        data = newData
    
    cname = 'CATALOG_NAME'
    catName = tableName#'test1.cat'
    confName = 'default_1.sex'
    nameLoc = data.find(cname) + len(cname)
    endLoc = data[nameLoc:].find('#') + nameLoc
    newData = data[:nameLoc] + ' ' +catName+ ' ' + data[endLoc:]
    data = newData

    cname = 'PARAMETERS_NAME'
    nameLoc = data.find(cname) + len(cname)
    endLoc = data[nameLoc:].find('#') + nameLoc
    newData = data[:nameLoc] + ' autodefault.param ' + data[endLoc:]
    
    #print(newData)
    with open(defaultDir+confName,'w') as f:
        f.write(newData)
    return(confName,tableName,catName)

In [ ]:
def findObjects(valList,image,attributeList):
    '''
    Changes config file and runs SExtractor on chosen image.
    
    Parameters:
        valList (lst): String values of SExtractor parameters to be inserted into the config file.
        image (str): Name of fits image to be SExtracted.
    
    Returns:
        tabTot (astropy table): Results of SExtractor.
    '''
    # note, the SExtractor parameters that will be changed are defined in the makeConfig function
    makeConfig(valList,attributeList=attributeList)
    tabTot =  sextractor(image,valList)
    return tabTot

In [ ]:
def saveSexTabasReg(table,regFileName='baseRegions.reg',regDirectory='/home/mj1e16/moleGazer/MoleGazer/regionDirectory/',aperture=15):
    '''
    Transform SExtractor output file to ds9 region file.
    
    Parameters:
        table (astropy table): SExtractor output table.
        regFileName (str): Name of ds9 region file.
        regDirectory (str): Directory of ds9 region file.
        aperture (int): Aperture size to be implemented in ds9 region file.
    '''
    # function does not return anything, just creates the region file
    xcoords = table['X_IMAGE'].tolist()
    ycoords = table['Y_IMAGE'].tolist()
    string = ''
    for x in range(len(xcoords)):
        string += 'circle({},{},{})\n'.format(xcoords[x],ycoords[x],aperture)
    with open(regDirectory+regFileName,'w') as f:
        f.write(string)

In [ ]:
def molePositions2Tab(molePositions):
    '''
    Create dataframe of mole x and y positions with upper and lower bounds for easy cross-matching.
    
    Parameters:
        molePositions (str): Name of DS9 region file containing mole positions.
        
    Returns:
        refPandasNew (pandas dataframe): Dataframe containing mole locations and upper/lower bounds.
    '''
    with open(molePositions,'r') as f:
        data = f.readlines()

    ycoord = []
    xcoord = []
    starLocations = []
    for x in data:
        if 'circle(' in x:
            locator1 = '('
            locator2 = ','
            loc1 = x.index(locator1) +1
            #print(x)
            loc2 = x[loc1:].index(locator2) + loc1 +1
            loc3 = x[loc2:].index(locator2) + loc2
            xcoord.append(x[loc1:loc2-1])
            ycoord.append(x[loc2:loc3])
            #starLocations.append([float(xcoord),float(ycoord)])
    tollerance = 5

    xUpper = [float(x) + tollerance for x in xcoord]
    xLower = [float(x) - tollerance for x in xcoord]
    yUpper = [float(x) + tollerance for x in ycoord]
    yLower = [float(x) - tollerance for x in ycoord]

    refDict = {'index':range(len(xcoord)),'xcoord':xcoord,'ycoord':ycoord,'xUpper':xUpper,'xLower':xLower,
              'yUpper':yUpper,'yLower':yLower}
    refPandasNew = pd.DataFrame(data=refDict)
    return refPandasNew

In [ ]:
def makeGraphfromFile(pyFile,valList,pyFileDirectory):
    '''
    Create 3-D graphs demonstrating the relationship between input parameters and completeness/accuracy.
    
    Parameters:
        pyFile (): Output file of SExtractor quality and input parameters.
        valList (): Values of SExtractor input parameters which were varied.
        pyFileDirectory (): Directory of Output file.
        
    
    '''
    
    subprocess.call(['cp',pyFileDirectory+pyFile,'./outputSextractorNew.py'])
    #pyFile = pyFile.replace('.py','')
    import outputSextractorNew as o
    
    outPutNecovered = o.Nrecovered
    outPutNtarget = o.Ntarget
    completeNess = [float(x)/float(y) if y != 0 else 0 for x, y in zip(outPutNecovered,outPutNtarget)]
    Nmax = max(o.Ntot)
    accuracy = [float(x)/float(Nmax) for x in o.Ntot]
    
    compPerFilter = [[] for x in range(len(valList[2]))]
    accPerfilter = [[] for x in range(len(valList[2]))]
    settingsPerFilter = [[] for x in range(len(valList[2]))]
    for x in range(len(valList[2])):
        for y in range(int(len(completeNess)/len(valList[2]))):
            #print((y*len(valList[2])+x))
            compPerFilter[x].append(completeNess[(y*len(valList[2])+x)])
            settingsPerFilter[x].append(o.settings[(y*len(valList[2])+x)])
            accPerfilter[x].append(accuracy[(y*len(valList[2])+x)])
            
    fullwhm = valList[0]
    threshold = valList[1]
    
    imageName = pyFile.replace('_results.py','')
    
    for y in range(len(valList[2])):
        accReformat = []
        compReformat = []
        for x in range(int(len(accPerfilter[y])/len(fullwhm))):
            accReformat.append(accPerfilter[y][x:x+len(fullwhm)])
            #print(accPerfilter[0][x:x+len(fullwhm)])
            compReformat.append(compPerFilter[y][x:x+len(fullwhm)])
            #totQualityReformat.append(tot[x:x+len(fullwhm)])

    #     fig = plt.figure(figsize=(15,10))
    #     ax = fig.gca(projection='3d')

        font = {'family' : 'serif',
                'weight' : 'normal',
                'size'   : 14}
        plt.rc('font', **font)
        plt.rc('axes',linewidth=1)
        #surf = ax.plot_surface(X,Y,Z,cmap='plasma',linewidth=0,antialiased=False)
        fig = plt.figure(figsize=(15,10))
        ax = fig.gca(projection='3d')
        X,Y = np.meshgrid(fullwhm,threshold)
        Z = np.array(accReformat)
        surf = ax.plot_surface(X,Y,Z,cmap='plasma',linewidth=0,antialiased=False)

        fig.colorbar(surf, shrink=0.5, aspect=5)
        #plt.gca().invert_zaxis()
        plt.xlabel('FWHM (Pixels)')
        plt.ylabel('Detection Threshold (Sigma)')
        ax.set_zlabel('Accuracy Score')
        plt.tight_layout()
        plt.savefig('/home/mj1e16/moleGazer/Photos/F/fitsImages/qualityCurves/acc_medium_filter_{}_image_{}_sextractor.png'.format(valList[2][y],imageName))
        plt.show()

    #     fig = plt.figure(figsize=(15,10))
    #     ax = fig.gca(projection='3d')

        font = {'family' : 'serif',
                'weight' : 'normal',
                'size'   : 14}
        plt.rc('font', **font)
        plt.rc('axes',linewidth=1)
        #surf = ax.plot_surface(X,Y,Z,cmap='plasma',linewidth=0,antialiased=False)
        fig = plt.figure(figsize=(15,10))
        ax = fig.gca(projection='3d')
        X,Y = np.meshgrid(fullwhm,threshold)
        Z = np.array(compReformat)
        surf = ax.plot_surface(X,Y,Z,cmap='plasma',linewidth=0,antialiased=False)

        fig.colorbar(surf, shrink=0.5, aspect=5)
        #plt.gca().invert_zaxis()
        plt.xlabel('FWHM (Pixels)')
        plt.ylabel('Detection Threshold (Sigma)')
        ax.set_zlabel('Completeness')
        plt.tight_layout()
        plt.savefig('/home/mj1e16/moleGazer/Photos/F/fitsImages/qualityCurves/comp_medium_filter_{}_image_{}_sextractor.png'.format(valList[2][y],imageName))
        plt.show()
    
        print(valList[2][y])

In [ ]:
sqlcode = '''
SELECT *
FROM pandasTable
JOIN refPandasNew
ON pandasTable.X_IMAGE < refPandasNew.xUpper
AND pandasTable.X_IMAGE > refPandasNew.xLower
AND pandasTable.Y_IMAGE < refPandasNew.yUpper
AND pandasTable.Y_IMAGE > refPandasNew.yLower
'''

In [ ]:
attributeList=['DETECT_THRESH','DETECT_MINAREA','FILTER_NAME'] # Name of SExtractor settings to be varied

In [ ]:
valList = [np.linspace(1,50,10),np.linspace(1,100,20),['default.conv','gauss_1.5_3x3.conv','gauss_2.0_3x3.conv','gauss_2.0_5x5.conv',
                                                    'gauss_2.5_5x5.conv','gauss_3.0_5x5.conv','gauss_3.0_7x7.conv',
                                                    'gauss_4.0_7x7.conv','gauss_5.0_9x9.conv','mexhat_1.5_5x5.conv',
                                                    'mexhat_2.0_7x7.conv','mexhat_2.5_7x7.conv','mexhat_3.0_9x9.conv',
                                                    'mexhat_4.0_9x9.conv','mexhat_5.0_11x11.conv','tophat_1.5_3x3.conv',
                                                    'tophat_2.0_3x3.conv','tophat_2.5_3x3.conv','tophat_3.0_3x3.conv',
                                                    'tophat_4.0_5x5.conv','tophat_5.0_5x5.conv']]
# values to be cycled for attributeList

In [ ]:
imageDirList = os.listdir(fitsImageDir)
sextractorReady = [x for x in imageDirList if '.fits' in x]
sextractorReady.sort()

# Only use one colour of fits file or a combination if they are all in the same directory use:
# sextractorReady = [x for x in sextractorReady if 'average' in x]

regList = [x for x in dirlist if '.reg' in x]
regList.sort()

In [ ]:
panTabs = []
for x in regList:
    panTabs.append(molePositions2Tab('/home/mj1e16/moleGazer/Photos/F/fitsImages/'+x)) # convert ds9 region files to table

In [ ]:
showImageWithResults([1,1,'default.conv'],'29005InvertedAverageCropped16.fits','/home/mj1e16/moleGazer/Photos/F/fitsImages/16bit/')

In [ ]:
t0 = time.time()

for imNum in range(len(sextractorReady)):
    t1 = time.time()
    Nrecovered = []
    Ntarget = []
    Ntot = []
    settings = []
    
    fakeHeadMoleIM = fitsImageDir+sextractorReady[imNum]
    refPandasNew = panTabs[imNum]
    
    for x in range(len(valList[0])):
        for y in range(len(valList[1])):
            for z in range(len(valList[2])):
                output = findObjects([valList[0][x],valList[1][y],valList[2][z]],fakeHeadMoleIM,attributeList)
                quality = assessQuality(output,sqlcode,refPandasNew)
                Nrecovered.append(quality[0])
                Ntarget.append(quality[1])
                Ntot.append(quality[2])
                settings.append([valList[0][x],valList[1][y],valList[2][z]])
    print(time.time() - t1)
    ogImNum = regList[imNum].replace('.reg','')
    with open(outputFileDir+'{}_results.py'.format(ogImNum),'w') as f:
        f.write('\nsettings = '+str(settings)+'\nNrecovered = '+str(Nrecovered)+'\nNtarget = '+str(Ntarget)+'\nNtot = '+str(Ntot))
        
print(time.time() -t0)

In [ ]:
makeGraphfromFile('29005_results.py',valList,'/home/mj1e16/moleGazer/Photos/F/fitsImages/simMolePos/')

In [ ]:
dirList = os.listdir(outputFileDir)
dirList = [x for x in dirList if '_results.py' in x]
print(dirList)

In [ ]:
for result in dirList:
    makeGraphfromFile(result,valList,'/home/mj1e16/moleGazer/Photos/F/fitsImages/simMolePos/')